<a href="https://colab.research.google.com/github/M20Jay/Data-Science-Portfolio/blob/main/House_price_prediction_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
"""
HOUSE PRICE PREDICTION
Multiple Linear Regression and Polynomial Regression

Dataset: House Price Regression Dataset
Features: Square_Footage, Num_Bedrooms, Num_Bathrooms, Year_Built,
          Lot_Size, Garage_Size, Neighborhood_Quality
Target: House_Price
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('default')
sns.set_palette("husl")

print("\n" + "="*70)
print("HOUSE PRICE PREDICTION - REGRESSION ANALYSIS")
print("="*70)


HOUSE PRICE PREDICTION - REGRESSION ANALYSIS


In [2]:
# ===============================================
# STEP 1: LOAD AND EXPLORE DATA
# ===============================================

In [3]:
print("\n" + "="*70)
print("STEP 1: DATA LOADING AND EXPLORATION")
print("="*70)


STEP 1: DATA LOADING AND EXPLORATION


In [7]:
# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Python Dataset/house_price_regression_dataset.csv')

print(f"\nDataset Shape: {df.shape}")
print(f"Total Records: {len(df)}")

print("\nColumn Names and Types:")
print(df.dtypes)

print("\nFirst 5 rows:")
print(df.head())

print("\nBasic Statistics:")
print(df.describe())

print("\nMissing Values:")
print(df.isnull().sum())

print("\nCorrelation with House Price:")
correlations = df.corr()['House_Price'].sort_values(ascending=False)
print(correlations)

# Ensure output directory exists
import os
output_dir = '/mnt/user-data/outputs/'
os.makedirs(output_dir, exist_ok=True)

# Visualize correlations
plt.figure(figsize=(10, 6))
correlations.drop('House_Price').plot(kind='barh', color='steelblue')
plt.xlabel('Correlation with House Price', fontsize=12, fontweight='bold')
plt.ylabel('Features', fontsize=12, fontweight='bold')
plt.title('Feature Correlation with House Price', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'correlation_analysis.png'), dpi=300, bbox_inches='tight')
print("\n✓ Saved: correlation_analysis.png")
plt.close()

# Visualize distributions
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, col in enumerate(df.columns):
    axes[i].hist(df[col], bins=30, color='steelblue', alpha=0.7, edgecolor='black')
    axes[i].set_title(col, fontweight='bold')
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Frequency')
    axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'data_distributions.png'), dpi=300, bbox_inches='tight')
print("✓ Saved: data_distributions.png")
plt.close()


Dataset Shape: (1000, 8)
Total Records: 1000

Column Names and Types:
Square_Footage            int64
Num_Bedrooms              int64
Num_Bathrooms             int64
Year_Built                int64
Lot_Size                float64
Garage_Size               int64
Neighborhood_Quality      int64
House_Price             float64
dtype: object

First 5 rows:
   Square_Footage  Num_Bedrooms  Num_Bathrooms  Year_Built  Lot_Size  \
0            1360             2              1        1981  0.599637   
1            4272             3              3        2016  4.753014   
2            3592             1              2        2016  3.634823   
3             966             1              2        1977  2.730667   
4            4926             2              1        1993  4.699073   

   Garage_Size  Neighborhood_Quality   House_Price  
0            0                     5  2.623829e+05  
1            1                     6  9.852609e+05  
2            0                     9  7.779774e+05  

In [8]:
# ===============================================
# STEP 2: DATA PREPROCESSING
# ===============================================


In [9]:
print("\n" + "="*70)
print("STEP 2: DATA PREPROCESSING")
print("="*70)

# Define features and target
X = df.drop('House_Price', axis=1)
y = df['House_Price']

print(f"\nFeatures shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature columns: {X.columns.tolist()}")

# Split data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nTraining set: {X_train.shape[0]} samples ({X_train.shape[0]/len(df)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(df)*100:.1f}%)")

# Standardize features (important for polynomial regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✓ Features standardized (mean=0, std=1)")



STEP 2: DATA PREPROCESSING

Features shape: (1000, 7)
Target shape: (1000,)

Feature columns: ['Square_Footage', 'Num_Bedrooms', 'Num_Bathrooms', 'Year_Built', 'Lot_Size', 'Garage_Size', 'Neighborhood_Quality']

Training set: 800 samples (80.0%)
Test set: 200 samples (20.0%)

✓ Features standardized (mean=0, std=1)


In [10]:
# ===============================================
# STEP 3: MULTIPLE LINEAR REGRESSION
# ===============================================

In [11]:
print("\n" + "="*70)
print("STEP 3: MULTIPLE LINEAR REGRESSION")
print("="*70)

# Train model
mlr_model = LinearRegression()
mlr_model.fit(X_train_scaled, y_train)

print("\n✓ Multiple Linear Regression model trained")

# Make predictions
y_pred_mlr_train = mlr_model.predict(X_train_scaled)
y_pred_mlr_test = mlr_model.predict(X_test_scaled)

# Calculate metrics
mlr_train_r2 = r2_score(y_train, y_pred_mlr_train)
mlr_test_r2 = r2_score(y_test, y_pred_mlr_test)
mlr_train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_mlr_train))
mlr_test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_mlr_test))
mlr_train_mae = mean_absolute_error(y_train, y_pred_mlr_train)
mlr_test_mae = mean_absolute_error(y_test, y_pred_mlr_test)

print("\n--- Multiple Linear Regression Performance ---")
print(f"Training R² Score:   {mlr_train_r2:.4f} ({mlr_train_r2*100:.2f}%)")
print(f"Test R² Score:       {mlr_test_r2:.4f} ({mlr_test_r2*100:.2f}%)")
print(f"Training RMSE:       ${mlr_train_rmse:,.2f}")
print(f"Test RMSE:           ${mlr_test_rmse:,.2f}")
print(f"Training MAE:        ${mlr_train_mae:,.2f}")
print(f"Test MAE:            ${mlr_test_mae:,.2f}")

# Feature coefficients
mlr_coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': mlr_model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print("\n--- Feature Coefficients (Standardized) ---")
print(mlr_coefficients.to_string(index=False))

# Visualize coefficients
plt.figure(figsize=(10, 6))
colors = ['green' if x > 0 else 'red' for x in mlr_coefficients['Coefficient']]
plt.barh(mlr_coefficients['Feature'], mlr_coefficients['Coefficient'], color=colors, alpha=0.7)
plt.xlabel('Coefficient Value (Standardized)', fontsize=12, fontweight='bold')
plt.ylabel('Feature', fontsize=12, fontweight='bold')
plt.title('Multiple Linear Regression - Feature Coefficients', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/mlr_coefficients.png', dpi=300, bbox_inches='tight')
print("\n✓ Saved: mlr_coefficients.png")
plt.close()

# Actual vs Predicted (MLR)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_mlr_test, alpha=0.6, color='steelblue', edgecolors='black', s=50)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
         'r--', linewidth=2, label='Perfect Prediction')
plt.xlabel('Actual House Price ($)', fontsize=12, fontweight='bold')
plt.ylabel('Predicted House Price ($)', fontsize=12, fontweight='bold')
plt.title(f'Multiple Linear Regression - Actual vs Predicted\nTest R² = {mlr_test_r2:.4f}',
          fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/mlr_actual_vs_predicted.png', dpi=300, bbox_inches='tight')
print("✓ Saved: mlr_actual_vs_predicted.png")
plt.close()


STEP 3: MULTIPLE LINEAR REGRESSION

✓ Multiple Linear Regression model trained

--- Multiple Linear Regression Performance ---
Training R² Score:   0.9985 (99.85%)
Test R² Score:       0.9984 (99.84%)
Training RMSE:       $9,687.65
Test RMSE:           $10,071.48
Training MAE:        $7,641.87
Test MAE:            $8,174.58

--- Feature Coefficients (Standardized) ---
             Feature   Coefficient
      Square_Footage 249787.914843
          Year_Built  20662.121612
            Lot_Size  19088.111033
        Num_Bedrooms  14524.734398
       Num_Bathrooms   6695.906775
         Garage_Size   4219.449419
Neighborhood_Quality    335.247488

✓ Saved: mlr_coefficients.png
✓ Saved: mlr_actual_vs_predicted.png


In [12]:
# ===============================================
# STEP 4: POLYNOMIAL REGRESSION (DEGREE 2)
# ===============================================

In [13]:
print("\n" + "="*70)
print("STEP 4: POLYNOMIAL REGRESSION (DEGREE 2)")
print("="*70)

# Create polynomial features (degree 2)
poly2 = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly2 = poly2.fit_transform(X_train_scaled)
X_test_poly2 = poly2.transform(X_test_scaled)

print(f"\nOriginal features: {X_train_scaled.shape[1]}")
print(f"Polynomial features (degree 2): {X_train_poly2.shape[1]}")
print(f"Feature expansion: {X_train_poly2.shape[1] - X_train_scaled.shape[1]} new features created")

# Train model
poly2_model = LinearRegression()
poly2_model.fit(X_train_poly2, y_train)

print("\n✓ Polynomial Regression (degree 2) model trained")

# Make predictions
y_pred_poly2_train = poly2_model.predict(X_train_poly2)
y_pred_poly2_test = poly2_model.predict(X_test_poly2)

# Calculate metrics
poly2_train_r2 = r2_score(y_train, y_pred_poly2_train)
poly2_test_r2 = r2_score(y_test, y_pred_poly2_test)
poly2_train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_poly2_train))
poly2_test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_poly2_test))
poly2_train_mae = mean_absolute_error(y_train, y_pred_poly2_train)
poly2_test_mae = mean_absolute_error(y_test, y_pred_poly2_test)

print("\n--- Polynomial Regression (Degree 2) Performance ---")
print(f"Training R² Score:   {poly2_train_r2:.4f} ({poly2_train_r2*100:.2f}%)")
print(f"Test R² Score:       {poly2_test_r2:.4f} ({poly2_test_r2*100:.2f}%)")
print(f"Training RMSE:       ${poly2_train_rmse:,.2f}")
print(f"Test RMSE:           ${poly2_test_rmse:,.2f}")
print(f"Training MAE:        ${poly2_train_mae:,.2f}")
print(f"Test MAE:            ${poly2_test_mae:,.2f}")

# Actual vs Predicted (Poly 2)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_poly2_test, alpha=0.6, color='orange', edgecolors='black', s=50)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
         'r--', linewidth=2, label='Perfect Prediction')
plt.xlabel('Actual House Price ($)', fontsize=12, fontweight='bold')
plt.ylabel('Predicted House Price ($)', fontsize=12, fontweight='bold')
plt.title(f'Polynomial Regression (Degree 2) - Actual vs Predicted\nTest R² = {poly2_test_r2:.4f}',
          fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/poly2_actual_vs_predicted.png', dpi=300, bbox_inches='tight')
print("\n✓ Saved: poly2_actual_vs_predicted.png")
plt.close()


STEP 4: POLYNOMIAL REGRESSION (DEGREE 2)

Original features: 7
Polynomial features (degree 2): 35
Feature expansion: 28 new features created

✓ Polynomial Regression (degree 2) model trained

--- Polynomial Regression (Degree 2) Performance ---
Training R² Score:   0.9986 (99.86%)
Test R² Score:       0.9984 (99.84%)
Training RMSE:       $9,594.55
Test RMSE:           $10,187.32
Training MAE:        $7,568.46
Test MAE:            $8,311.02

✓ Saved: poly2_actual_vs_predicted.png


In [14]:
# ===============================================
# STEP 5: POLYNOMIAL REGRESSION (DEGREE 3)
# ===============================================


In [15]:
print("\n" + "="*70)
print("STEP 5: POLYNOMIAL REGRESSION (DEGREE 3)")
print("="*70)

# Create polynomial features (degree 3)
poly3 = PolynomialFeatures(degree=3, include_bias=False)
X_train_poly3 = poly3.fit_transform(X_train_scaled)
X_test_poly3 = poly3.transform(X_test_scaled)

print(f"\nOriginal features: {X_train_scaled.shape[1]}")
print(f"Polynomial features (degree 3): {X_train_poly3.shape[1]}")
print(f"Feature expansion: {X_train_poly3.shape[1] - X_train_scaled.shape[1]} new features created")

# Train model
poly3_model = LinearRegression()
poly3_model.fit(X_train_poly3, y_train)

print("\n✓ Polynomial Regression (degree 3) model trained")

# Make predictions
y_pred_poly3_train = poly3_model.predict(X_train_poly3)
y_pred_poly3_test = poly3_model.predict(X_test_poly3)

# Calculate metrics
poly3_train_r2 = r2_score(y_train, y_pred_poly3_train)
poly3_test_r2 = r2_score(y_test, y_pred_poly3_test)
poly3_train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_poly3_train))
poly3_test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_poly3_test))
poly3_train_mae = mean_absolute_error(y_train, y_pred_poly3_train)
poly3_test_mae = mean_absolute_error(y_test, y_pred_poly3_test)

print("\n--- Polynomial Regression (Degree 3) Performance ---")
print(f"Training R² Score:   {poly3_train_r2:.4f} ({poly3_train_r2*100:.2f}%)")
print(f"Test R² Score:       {poly3_test_r2:.4f} ({poly3_test_r2*100:.2f}%)")
print(f"Training RMSE:       ${poly3_train_rmse:,.2f}")
print(f"Test RMSE:           ${poly3_test_rmse:,.2f}")
print(f"Training MAE:        ${poly3_train_mae:,.2f}")
print(f"Test MAE:            ${poly3_test_mae:,.2f}")

# Actual vs Predicted (Poly 3)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_poly3_test, alpha=0.6, color='green', edgecolors='black', s=50)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
         'r--', linewidth=2, label='Perfect Prediction')
plt.xlabel('Actual House Price ($)', fontsize=12, fontweight='bold')
plt.ylabel('Predicted House Price ($)', fontsize=12, fontweight='bold')
plt.title(f'Polynomial Regression (Degree 3) - Actual vs Predicted\nTest R² = {poly3_test_r2:.4f}',
          fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/poly3_actual_vs_predicted.png', dpi=300, bbox_inches='tight')
print("\n✓ Saved: poly3_actual_vs_predicted.png")
plt.close()


STEP 5: POLYNOMIAL REGRESSION (DEGREE 3)

Original features: 7
Polynomial features (degree 3): 119
Feature expansion: 112 new features created

✓ Polynomial Regression (degree 3) model trained

--- Polynomial Regression (Degree 3) Performance ---
Training R² Score:   0.9987 (99.87%)
Test R² Score:       0.9981 (99.81%)
Training RMSE:       $9,113.28
Test RMSE:           $11,152.35
Training MAE:        $7,204.01
Test MAE:            $8,932.69

✓ Saved: poly3_actual_vs_predicted.png


In [16]:
# ===============================================
# STEP 6: MODEL COMPARISON
# ===============================================


In [17]:
print("\n" + "="*70)
print("STEP 6: MODEL COMPARISON")
print("="*70)

# Create comparison table
comparison_df = pd.DataFrame({
    'Model': ['Multiple Linear Regression', 'Polynomial Regression (Degree 2)',
              'Polynomial Regression (Degree 3)'],
    'Train R²': [mlr_train_r2, poly2_train_r2, poly3_train_r2],
    'Test R²': [mlr_test_r2, poly2_test_r2, poly3_test_r2],
    'Train RMSE': [mlr_train_rmse, poly2_train_rmse, poly3_train_rmse],
    'Test RMSE': [mlr_test_rmse, poly2_test_rmse, poly3_test_rmse],
    'Train MAE': [mlr_train_mae, poly2_train_mae, poly3_train_mae],
    'Test MAE': [mlr_test_mae, poly2_test_mae, poly3_test_mae],
    'Num Features': [X_train_scaled.shape[1], X_train_poly2.shape[1], X_train_poly3.shape[1]]
})

print("\n" + comparison_df.to_string(index=False))

# Identify best model
best_model_idx = comparison_df['Test R²'].idxmax()
best_model_name = comparison_df.loc[best_model_idx, 'Model']
best_test_r2 = comparison_df.loc[best_model_idx, 'Test R²']

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   Test R² Score: {best_test_r2:.4f} ({best_test_r2*100:.2f}%)")

# Calculate improvements
mlr_to_poly2 = ((poly2_test_r2 - mlr_test_r2) / mlr_test_r2) * 100
mlr_to_poly3 = ((poly3_test_r2 - mlr_test_r2) / mlr_test_r2) * 100

print(f"\n--- Performance Improvements over Multiple Linear Regression ---")
print(f"Polynomial (Degree 2): {mlr_to_poly2:+.2f}%")
print(f"Polynomial (Degree 3): {mlr_to_poly3:+.2f}%")

# Visualize R² comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# R² Comparison
models = ['MLR', 'Poly\n(Deg 2)', 'Poly\n(Deg 3)']
train_r2_scores = [mlr_train_r2, poly2_train_r2, poly3_train_r2]
test_r2_scores = [mlr_test_r2, poly2_test_r2, poly3_test_r2]

x_pos = np.arange(len(models))
width = 0.35

axes[0].bar(x_pos - width/2, train_r2_scores, width, label='Training R²', alpha=0.8)
axes[0].bar(x_pos + width/2, test_r2_scores, width, label='Test R²', alpha=0.8)
axes[0].set_ylabel('R² Score', fontweight='bold')
axes[0].set_title('R² Score Comparison', fontweight='bold', fontsize=12)
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(models)
axes[0].legend()
axes[0].set_ylim(0, 1)
axes[0].grid(axis='y', alpha=0.3)

# RMSE Comparison
test_rmse_scores = [mlr_test_rmse, poly2_test_rmse, poly3_test_rmse]
axes[1].bar(models, test_rmse_scores, color=['steelblue', 'orange', 'green'], alpha=0.8)
axes[1].set_ylabel('RMSE ($)', fontweight='bold')
axes[1].set_title('RMSE Comparison (Test Set)', fontweight='bold', fontsize=12)
axes[1].grid(axis='y', alpha=0.3)
for i, v in enumerate(test_rmse_scores):
    axes[1].text(i, v + 500, f'${v:,.0f}', ha='center', fontweight='bold', fontsize=9)

# MAE Comparison
test_mae_scores = [mlr_test_mae, poly2_test_mae, poly3_test_mae]
axes[2].bar(models, test_mae_scores, color=['steelblue', 'orange', 'green'], alpha=0.8)
axes[2].set_ylabel('MAE ($)', fontweight='bold')
axes[2].set_title('MAE Comparison (Test Set)', fontweight='bold', fontsize=12)
axes[2].grid(axis='y', alpha=0.3)
for i, v in enumerate(test_mae_scores):
    axes[2].text(i, v + 500, f'${v:,.0f}', ha='center', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/model_comparison.png', dpi=300, bbox_inches='tight')
print("\n✓ Saved: model_comparison.png")
plt.close()


STEP 6: MODEL COMPARISON

                           Model  Train R²  Test R²  Train RMSE    Test RMSE   Train MAE    Test MAE  Num Features
      Multiple Linear Regression  0.998538 0.998426 9687.648422 10071.484424 7641.867910 8174.583600             7
Polynomial Regression (Degree 2)  0.998566 0.998390 9594.545155 10187.316600 7568.459928 8311.015875            35
Polynomial Regression (Degree 3)  0.998706 0.998070 9113.283509 11152.351272 7204.005210 8932.686335           119

🏆 BEST MODEL: Multiple Linear Regression
   Test R² Score: 0.9984 (99.84%)

--- Performance Improvements over Multiple Linear Regression ---
Polynomial (Degree 2): -0.00%
Polynomial (Degree 3): -0.04%

✓ Saved: model_comparison.png


In [18]:
# ===============================================
# STEP 7: RESIDUAL ANALYSIS
# ===============================================


In [19]:
print("\n" + "="*70)
print("STEP 7: RESIDUAL ANALYSIS")
print("="*70)

# Calculate residuals
mlr_residuals = y_test - y_pred_mlr_test
poly2_residuals = y_test - y_pred_poly2_test
poly3_residuals = y_test - y_pred_poly3_test

# Plot residuals
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# MLR Residuals
axes[0].scatter(y_pred_mlr_test, mlr_residuals, alpha=0.6, color='steelblue', edgecolors='black')
axes[0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel('Predicted Price ($)', fontweight='bold')
axes[0].set_ylabel('Residuals ($)', fontweight='bold')
axes[0].set_title('MLR - Residual Plot', fontweight='bold', fontsize=12)
axes[0].grid(alpha=0.3)

# Poly 2 Residuals
axes[1].scatter(y_pred_poly2_test, poly2_residuals, alpha=0.6, color='orange', edgecolors='black')
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Predicted Price ($)', fontweight='bold')
axes[1].set_ylabel('Residuals ($)', fontweight='bold')
axes[1].set_title('Polynomial (Deg 2) - Residual Plot', fontweight='bold', fontsize=12)
axes[1].grid(alpha=0.3)

# Poly 3 Residuals
axes[2].scatter(y_pred_poly3_test, poly3_residuals, alpha=0.6, color='green', edgecolors='black')
axes[2].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[2].set_xlabel('Predicted Price ($)', fontweight='bold')
axes[2].set_ylabel('Residuals ($)', fontweight='bold')
axes[2].set_title('Polynomial (Deg 3) - Residual Plot', fontweight='bold', fontsize=12)
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/residual_analysis.png', dpi=300, bbox_inches='tight')
print("\n✓ Saved: residual_analysis.png")
plt.close()

print("\n--- Residual Statistics ---")
print(f"MLR - Mean Residual: ${mlr_residuals.mean():,.2f} (should be ~0)")
print(f"MLR - Std Residual:  ${mlr_residuals.std():,.2f}")
print(f"\nPoly 2 - Mean Residual: ${poly2_residuals.mean():,.2f} (should be ~0)")
print(f"Poly 2 - Std Residual:  ${poly2_residuals.std():,.2f}")
print(f"\nPoly 3 - Mean Residual: ${poly3_residuals.mean():,.2f} (should be ~0)")
print(f"Poly 3 - Std Residual:  ${poly3_residuals.std():,.2f}")


STEP 7: RESIDUAL ANALYSIS

✓ Saved: residual_analysis.png

--- Residual Statistics ---
MLR - Mean Residual: $-118.92 (should be ~0)
MLR - Std Residual:  $10,096.05

Poly 2 - Mean Residual: $13.84 (should be ~0)
Poly 2 - Std Residual:  $10,212.87

Poly 3 - Mean Residual: $-521.76 (should be ~0)
Poly 3 - Std Residual:  $11,168.09


In [20]:
# ===============================================
# STEP 8: PREDICTION EXAMPLE
# ===============================================

In [21]:
print("\n" + "="*70)
print("STEP 8: PREDICTION EXAMPLE")
print("="*70)

# Create sample house
sample_house = pd.DataFrame({
    'Square_Footage': [2500],
    'Num_Bedrooms': [3],
    'Num_Bathrooms': [2],
    'Year_Built': [2010],
    'Lot_Size': [8000.0],
    'Garage_Size': [2],
    'Neighborhood_Quality': [7]
})

print("\nSample House Features:")
print(sample_house.to_string(index=False))

# Standardize
sample_scaled = scaler.transform(sample_house)

# Create polynomial features
sample_poly2 = poly2.transform(sample_scaled)
sample_poly3 = poly3.transform(sample_scaled)

# Make predictions
pred_mlr = mlr_model.predict(sample_scaled)[0]
pred_poly2 = poly2_model.predict(sample_poly2)[0]
pred_poly3 = poly3_model.predict(sample_poly3)[0]

print("\n--- Price Predictions ---")
print(f"Multiple Linear Regression:      ${pred_mlr:,.2f}")
print(f"Polynomial Regression (Deg 2):   ${pred_poly2:,.2f}")
print(f"Polynomial Regression (Deg 3):   ${pred_poly3:,.2f}")
print(f"\nAverage Prediction:              ${np.mean([pred_mlr, pred_poly2, pred_poly3]):,.2f}")
print(f"Prediction Range:                ${min(pred_mlr, pred_poly2, pred_poly3):,.2f} - ${max(pred_mlr, pred_poly2, pred_poly3):,.2f}")


STEP 8: PREDICTION EXAMPLE

Sample House Features:
 Square_Footage  Num_Bedrooms  Num_Bathrooms  Year_Built  Lot_Size  Garage_Size  Neighborhood_Quality
           2500             3              2        2010    8000.0            2                     7

--- Price Predictions ---
Multiple Linear Regression:      $119,626,490.98
Polynomial Regression (Deg 2):   $-5,894,278,572.06
Polynomial Regression (Deg 3):   $63,082,990,627,054.59

Average Prediction:              $21,025,738,658,324.51
Prediction Range:                $-5,894,278,572.06 - $63,082,990,627,054.59


In [22]:
# ===============================================
# STEP 9: SAVE RESULTS
# ===============================================

In [23]:
print("\n" + "="*70)
print("STEP 9: SAVING RESULTS")
print("="*70)

# Save comparison to Excel
with pd.ExcelWriter('/mnt/user-data/outputs/house_price_results.xlsx', engine='openpyxl') as writer:
    # Model comparison
    comparison_df.to_excel(writer, sheet_name='Model Comparison', index=False)

    # Feature coefficients
    mlr_coefficients.to_excel(writer, sheet_name='MLR Coefficients', index=False)

    # Predictions sample
    predictions_df = pd.DataFrame({
        'Actual': y_test.values,
        'MLR_Predicted': y_pred_mlr_test,
        'Poly2_Predicted': y_pred_poly2_test,
        'Poly3_Predicted': y_pred_poly3_test,
        'MLR_Error': mlr_residuals.values,
        'Poly2_Error': poly2_residuals.values,
        'Poly3_Error': poly3_residuals.values
    })
    predictions_df.to_excel(writer, sheet_name='Predictions', index=False)

    # Sample prediction
    sample_pred_df = pd.DataFrame({
        'Model': ['Multiple Linear Regression', 'Polynomial (Degree 2)', 'Polynomial (Degree 3)'],
        'Predicted_Price': [pred_mlr, pred_poly2, pred_poly3]
    })
    sample_pred_df.to_excel(writer, sheet_name='Sample Prediction', index=False)

print("\n✓ Saved: house_price_results.xlsx")


STEP 9: SAVING RESULTS

✓ Saved: house_price_results.xlsx


In [24]:
# ===============================================
# SUMMARY
# ===============================================


In [25]:
print("\n" + "="*70)
print("ANALYSIS COMPLETE!")
print("="*70)

print("\nGenerated Files:")
print("  1. correlation_analysis.png")
print("  2. data_distributions.png")
print("  3. mlr_coefficients.png")
print("  4. mlr_actual_vs_predicted.png")
print("  5. poly2_actual_vs_predicted.png")
print("  6. poly3_actual_vs_predicted.png")
print("  7. model_comparison.png")
print("  8. residual_analysis.png")
print("  9. house_price_results.xlsx")

print("\n🏆 BEST MODEL:", best_model_name)
print(f"   Test R² Score: {best_test_r2:.4f} ({best_test_r2*100:.2f}%)")
print(f"   Test RMSE: ${comparison_df.loc[best_model_idx, 'Test RMSE']:,.2f}")
print(f"   Test MAE: ${comparison_df.loc[best_model_idx, 'Test MAE']:,.2f}")

print("\nKEY FINDINGS:")
if poly3_test_r2 > poly2_test_r2 > mlr_test_r2:
    print("  • Polynomial features significantly improve prediction accuracy")
    print("  • Higher degree polynomials capture non-linear relationships better")
elif poly2_test_r2 > mlr_test_r2 and poly3_test_r2 < poly2_test_r2:
    print("  • Polynomial (Degree 2) provides best balance")
    print("  • Degree 3 may be overfitting the training data")
else:
    print("  • Multiple Linear Regression performs adequately")
    print("  • Polynomial features provide marginal improvement")

print("\n" + "="*70)



ANALYSIS COMPLETE!

Generated Files:
  1. correlation_analysis.png
  2. data_distributions.png
  3. mlr_coefficients.png
  4. mlr_actual_vs_predicted.png
  5. poly2_actual_vs_predicted.png
  6. poly3_actual_vs_predicted.png
  7. model_comparison.png
  8. residual_analysis.png
  9. house_price_results.xlsx

🏆 BEST MODEL: Multiple Linear Regression
   Test R² Score: 0.9984 (99.84%)
   Test RMSE: $10,071.48
   Test MAE: $8,174.58

KEY FINDINGS:
  • Multiple Linear Regression performs adequately
  • Polynomial features provide marginal improvement

